In [3]:
!pip show kfp

Name: kfp
Version: 2.7.0
Summary: Kubeflow Pipelines SDK
Home-page: https://github.com/kubeflow/pipelines
Author: The Kubeflow Authors
Author-email: 
License: 
Location: /Applications/anaconda3/lib/python3.11/site-packages
Requires: click, docstring-parser, google-api-core, google-auth, google-cloud-storage, kfp-pipeline-spec, kfp-server-api, kubernetes, protobuf, PyYAML, requests-toolbelt, tabulate, urllib3
Required-by: 


In [9]:
import kfp
import kfp.components as comp
import requests
import kfp.dsl as dsl

In [3]:
@dsl.component()
def prepare_data():
    import pandas as pd
    df = pd.read_csv('iris.csv')
    df = df.dropna()
    df.to_csv(f'data/final_df.csv', index=False)
    print("\n ---- data csv is saved to PV location /data/final_df.csv ----")

In [4]:
def train_test_split():
    import pandas as pd
    from sklearn.model_selection import train_test_split
    import numpy as np
    print("---- Inside train_test_split component ----")
    final_data = pd.read_csv(f'data/final_df.csv')
    X = final_data.drop(columns='class')
    y = final_data['class']
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=47)
    np.save(f'data/X_train.npy', X_train)
    np.save(f'data/X_train.npy', X_test)
    np.save(f'data/X_train.npy', y_train)
    np.save(f'data/X_train.npy', y_test)
    print("\n---- X_train ----")
    print("\n")
    print(X_train)
    
    print("\n---- X_test ----")
    print("\n")
    print(X_test)
    
    print("\n---- y_train ----")
    print("\n")
    print(y_train)
    
    print("\n---- y_test ----")
    print("\n")
    print(y_test)

In [5]:
def train_basic_classifier():
    from sklearn.linear_model import LogisticRegression
    import numpy as np
    import pandas as pd

    print("---- Inside training_basic_classifier component ----")

    X_train = np.load(f'data/X_train.npy')
    y_train = np.load(f'data/y_train.npy')

    classifier = LogisticRegression(max_iter=500)
    classifier.fit(X_train,y_train)

    import pickle
    with open(f'data/model.pkl','wb') as f:
        pickle.dump(classifier,f)

    print("\n logistic regression classifier is trained on iris data and saved to PV location /data/model.pkl ----")

In [6]:
def predict_on_test_data():
    import pandas as pd
    import numpy as np
    import pickle
    print("---- Inside predict_on_test_data component ----")
    with open(f'data/model.pkl','rb') as f:
        logistic_reg_model = pickle.load(f)

    X_test = np.load(f'data/X_test')
    y_pred = logistic_reg_model.predict(X_test)
    np.save(f'data/y_pred.npy', y_pred)

    print("\n---- Predicted classes ----")
    print("\n")
    print(y_pred)

In [7]:
def predict_prob_on_test_data():
    import pandas as pd
    import numpy as np
    import pickle
    print("---- Inside predict_prob_on_test_data component ----")
    with open(f'data/model.pkl','rb') as f:
        logistic_reg_model = pickle.load(f)
    X_test = np.load(f'data/X_test.npy',allow_pickle=True)
    y_pred_prob = logistic_reg_model.predict_proba(X_test)
    np.save(f'data/y_pred_prob.npy', y_pred_prob)
    
    print("\n---- Predicted Probabilities ----")
    print("\n")
    print(y_pred_prob)

In [20]:
def get_metrics():
    import pandas as pd
    import numpy as np
    from sklearn.metrics import accuracy_score,precision_score,recall_score,log_loss,classification_report
    print("---- Inside get_metrics component ----")
    y_test = np.load(f'data/y_test.npy')
    y_pred = np.load(f'data/y_pred.npy')
    y_pred_prob = np.load(f'data/y_pred_prob.npy')
    acc = accuracy_score(y_pred,y_test)
    prec = precision_score(y_pred,y_test)
    recall = recall_score(y_pred,y_test)
    entropy = log_loss(y_test, y_pred_prob)
    print(classification_report(y_pred,y_test))
    print({'Accuracy':round(acc,2),'precision': round(prec, 2), 'recall': round(recall, 2), 'entropy': round(entropy, 2)})

# Kubeflow pipeline creation work start from here

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install kfp

  Using cached kfp-2.7.0.tar.gz (441 kB)
  Preparing metadata (setup.py) ... done
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_core-2.19.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_auth-2.30.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached google_cloud_storage-2.17.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached kfp_pipeline_spec-0.3.0-py3-none-any.whl.metadata (329 bytes)
  Using cached kfp-server-api-2.0.5.tar.gz (63 kB)
  Preparing metadata (setup.py) ... done
  Using cached kubernetes-26.1.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl.metadata (14 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached googleapis_common_protos-1.63.1-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached goo

In [5]:
pip install jupyter pandas

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached types_python_dateutil-2.9.0.20240316-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 5.9 MB/s eta 0:

In [11]:
import kfp
import kfp.components as comp
import requests
import kfp.dsl as dsl

In [13]:
create_step_prepare_data = comp.(
    func=prepare_data,
    base_image='python:3.7',
    packages_to_install=['pandas==1.2.4','numpy==1.21.0']
)

AttributeError: module 'kfp.components' has no attribute 'create_step_prepare_data'

In [35]:
!pip install kfp --upgrade